In [ ]:
import yfinance as yf
import pandas as pd

In [ ]:
# Ativos
ativos = ["HGRU11.SA", "HSLG11.SA", "KNCR11.SA", "MXRF11.SA", "PORD11.SA",
          "RZTR11.SA", "VGHF11.SA", "GGRC11.SA", "XPLG11.SA", "XPML11.SA",
          "GARE11.SA", "VISC11.SA"]

# Baixando os dados
data = yf.download(ativos, period="max")

# Selecionando apenas as colunas necessárias
data = data['Close']

# Formatar o índice (data) para o formato desejado
data.index = data.index.strftime('%d-%m-%Y')

# Usando a função melt para transformar o DataFrame
df_melted = data.melt(ignore_index=False, var_name='ticker', value_name='valor_cotação')

# Renomeando o índice para 'date'
df_melted.index.name = 'date'

# Resetando o índice para tornar 'date' uma coluna
df_melted = df_melted.reset_index()

# Reorganizando as colunas na ordem desejada
df_organizado = df_melted[['date', 'ticker', 'valor_cotação']]

# Salvar o DataFrame em um arquivo Excel
df_organizado.to_excel('dados_organizados.xlsx', index=False)

df_organizado.head()